In [457]:
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
import requests
import time
from datetime import datetime
import math
import numpy as np

In [458]:
stock_tickers_dirty = []
stock_prices_dirty = []

In [459]:
stock_tickers = []
stock_prices = []
stock_industries = []
stock_PE = []
stock_percs = []

In [460]:
removed_stocks = 0
processed_stocks = 0

driver = webdriver.Chrome()
driver.get("https://finance.yahoo.com/most-active?count=100")
#open filters
driver.find_element_by_xpath("/html/body/div/div/div/div/div/div[2]/div/div/div[5]/div/div/div/div[2]/header/button").click()
#click small cap
driver.find_element_by_xpath("/html/body/div/div/div/div/div/div[2]/div/div/div[5]/div/div/div/div[2]/div/div/div[2]/div/div[2]/div/button[1]").click()
#clear volume text box
driver.find_element_by_xpath("/html/body/div/div/div/div/div/div[2]/div/div/div[5]/div/div/div/div[2]/div/div/div[3]/div/div[2]/input").clear()
#1 mill volume
driver.find_element_by_xpath("/html/body/div/div/div/div/div/div[2]/div/div/div[5]/div/div/div/div[2]/div/div/div[3]/div/div[2]/input").send_keys("500000")

time.sleep(1)
#click "find stocks"
driver.find_element_by_xpath("/html/body/div/div/div/div/div/div[2]/div/div/div[5]/div/div/div/div[2]/div[1]/div[3]/button[1]").click()


time.sleep(1)
bigsoup = bs(driver.page_source,"html.parser")
resultstring = bigsoup.find(class_="Mstart(15px) Fw(500) Fz(s)").string
resultnum = int(resultstring[9:-8])
pagesnum = math.ceil(resultnum/100)


for page in range(pagesnum):
    bigsoup = bs(driver.page_source,"html.parser")
    tickerhtml = bigsoup.tbody.find_all(class_ = "Fw(600) C($linkColor)")
    pricehtml = bigsoup.tbody.find_all(attrs={"aria-label":"Price (Intraday)"})
    
    for tickerraw in tickerhtml:
        ticker = tickerraw.string
        pricestring = (pricehtml[tickerhtml.index(tickerraw)].string)
        pricedirty = pricestring.replace(',','')
        price = round(float(pricedirty), 2)
        #add values to lists
        if price >= .5:
            stock_tickers_dirty.append(ticker)
            stock_prices_dirty.append(price)
        else:
            removed_stocks += 1
            
        processed_stocks += 1
        
    nextpage = driver.find_element_by_xpath("html/body/div/div/div/div/div/div[2]/div/div/div[6]/div/div/section/div/div[2]/div[2]/button[3]")
    
    if processed_stocks < resultnum:
        nextpage.click()
        time.sleep(1)
    else: break
        
print("Done!")
print(f"Search returned {resultnum} results, {removed_stocks} were omitted.")
print("Total pages:", pagesnum)
print("Stocks count:", len(stock_tickers_dirty))

Done!
Search returned 2783 results, 721 were omitted.
Total pages: 28
Stocks count: 2062


In [461]:
def calcperatio(stock, soup):
    earningsraw = soup.body.find(attrs={"data-test":"EPS_RATIO-value"})
    if earningsraw is None or earningsraw.text == "N/A":
        peratio = "N/A"
        return peratio
    
    earningstext = earningsraw.text
    earnings = float(earningstext.replace(',',''))
    
    if earnings == 0:
        peratio = 0
    else: 
        peratio = round(tickerprice/earnings, 2)
    return peratio

count = 1
countremoved = 0

for stock in stock_tickers_dirty:
    website = f"https://finance.yahoo.com/quote/{stock}?p={stock}"
    webrequest = requests.get(website).text
    soup = bs(webrequest, "html.parser")
    
    tickerindex = stock_tickers_dirty.index(stock)
    tickerprice = round(float(stock_prices_dirty[tickerindex]),2)
    
#find industry   
    industryraw = soup.body.find(class_ = "D(ib) Va(t)")
    if industryraw is None or industryraw == "N/A":
        industry = "N/A"
    else:
        industrytext = industryraw.text
        industryspace = industrytext.replace(u'\xa0', u' ')
        industryrough = industryspace.split("Industry: ")
        industry = industryrough[1].split("Full Time")[0]
        
    if industry == "":
        industry = "N/A"

#find price/earning ratio
    peratioraw = soup.body.find(attrs={"data-test":"PE_RATIO-value"})
        
    if peratioraw:
        if peratioraw.text != "N/A":
            peratiotext = peratioraw.text
            peratioclean = peratiotext.replace(',','')
            peratio = float(peratioclean)
        else:
            peratio = calcperatio(stock, soup)
    else:
        peratio = calcperatio(stock, soup)
    
#find intraday percentage change
    percridnextraw_day = soup.body.find(id = "quote-market-notice")
    if percridnextraw_day:
        percridnext_day = percridnextraw_day["data-reactid"]
    else:
        percridnext_day = 51
        
    percrid_day = int(percridnext_day) - 1
    
    percentageraw_day = soup.body.find(class_ = "Trsdu(0.3s) Fw(500) Pstart(10px) Fz(24px) C($positiveColor)", attrs={"data-reactid":"{}".format(percrid_day)})
    if percentageraw_day is None:
        percentageraw_day = soup.body.find(class_ = "Trsdu(0.3s) Fw(500) Pstart(10px) Fz(24px) C($negativeColor)", attrs={"data-reactid":"{}".format(percrid_day)})
        if percentageraw_day is None:
            percentageraw_day = soup.body.find(class_ = "Trsdu(0.3s) Fw(500) Pstart(10px) Fz(24px)", attrs={"data-reactid":"{}".format(percrid_day)})
            if percentageraw_day is None:
                percentageraw_day = 0
                
    if percentageraw_day == 0:
        percentage_day = percentageraw_day
    else:
        percentagedirty_day = percentageraw_day.text
        percentagestring_day = percentagedirty_day.split(" ")[1][1:-2]
        percentage_day = float(percentagestring_day.replace(',',''))

#find aftermarket percentage change
    percridnext_night_raw = soup.body.find(class_ = "C($tertiaryColor) Fz(12px) smartphone_Fz(xs)")
    
    if percridnext_night_raw:
        percridnext_night = percridnext_night_raw["data-reactid"]
        percrid_night = int(percridnext_night) - 2
        
        percentageraw_night = soup.body.find(class_ = "Trsdu(0.3s) Mstart(4px) D(ib) Fz(24px) C($positiveColor)", attrs={"data-reactid":"{}".format(percrid_night)})
        if percentageraw_night is None:
            percentageraw_night = soup.body.find(class_ = "Trsdu(0.3s) Mstart(4px) D(ib) Fz(24px) C($negativeColor)", attrs={"data-reactid":"{}".format(percrid_night)})
            if percentageraw_night is None:
                percentageraw_night = soup.body.find(class_ = "Trsdu(0.3s) Mstart(4px) D(ib) Fz(24px)", attrs={"data-reactid":"{}".format(percrid_night)})
        
        percentagedirty_night = percentageraw_night.text
        percentagestring_night = percentagedirty_night.split(" ")[1][1:-2]
        percentage_night = float(percentagestring_night.replace(',',''))
    else:
        percentage_night = 0
            
    percentage = round(percentage_day + percentage_night, 2)
    
# percentage
    if (count/50).is_integer():
        dirtyloading = (count/len(stock_tickers_dirty))*100
        print(round(dirtyloading, 1), end = '%\n')

#remove bad data
    if industry == "N/A" or peratio == "N/A" or percentage == "N/A":
        print(f"{stock} Removed")
        countremoved += 1
        count += 1
        continue

    
#append lists
    else:
        stock_tickers.append(stock)
        stock_prices.append(tickerprice)
        stock_industries.append(industry)
        stock_PE.append(peratio)
        stock_percs.append(percentage)
        count+= 1
    
print(f"Done! {countremoved} stocks removed, {len(stock_tickers)} cleaned results.")

2.4%
SNR Removed
4.8%
7.3%
HST Removed
GBTC Removed
9.7%
ACGL Removed
CIG Removed
12.1%
JXN Removed
14.5%
JOBY Removed
17.0%
BRFS Removed
PSLV Removed
19.4%
AXTA Removed
DNB Removed
FOXA Removed
21.8%
CIK Removed
NWSA Removed
24.2%
ATAX Removed
CANO Removed
AMX Removed
NLSN Removed
26.7%
PAGP Removed
HTA Removed
29.1%
IBN Removed
FNB Removed
31.5%
PRMW Removed
AMBP Removed
ESRT Removed
LBTYK Removed
33.9%
AMH Removed
SHO Removed
NSL Removed
36.4%
EQC Removed
SRUUF Removed
38.8%
AGC Removed
NUAN Removed
STL Removed
SLM Removed
LBTYA Removed
UDR Removed
41.2%
PGRE Removed
BRX Removed
ALSN Removed
BDN Removed
OSH Removed
BRLT Removed
DCRC Removed
REXR Removed
43.6%
LW Removed
DRE Removed
LXP Removed
CONX Removed
OWL Removed
EQH Removed
ASB Removed
ESI Removed
TWNK Removed
46.1%
USFD Removed
ES Removed
SCI Removed
PV Removed
NWG Removed
SRC Removed
48.5%
KCAC Removed
GPK Removed
GASNY Removed
CGXEF Removed
WBT Removed
SNV Removed
EQR Removed
CHK Removed
DEI Removed
HR Removed
LKQ Removed
5

In [462]:
stockinfo = {"Stock Tickers":stock_tickers, "Industry":stock_industries, "Price":stock_prices, "Percent Change (Today)":stock_percs, "P/E Ratio": stock_PE}

stockdf = pd.DataFrame(stockinfo)

In [463]:
stockdf['Frequency'] = stockdf.groupby(['Industry'])['P/E Ratio'].transform('count')

In [464]:
threshold = 20
stockdf2 = stockdf.drop(stockdf[stockdf.Frequency<threshold].index)

industryNumResults = eval(input("How many industries would you like me to display?"))
stockNumResults = eval(input("How many stocks per industry would you like me to display?"))

topindustries = stockdf2.groupby("Industry")[["Percent Change (Today)"]].mean().sort_values("Percent Change (Today)", ascending = False).head(industryNumResults).reset_index()

industryresults = topindustries["Industry"]
percresults = topindustries["Percent Change (Today)"]

for x in range(industryNumResults):
    industry = industryresults[x]
    percentage = percresults[x]
    
    print(x+1, end = ": ")
    
    if percentage >= 0:
        print(f"{industry} (+{round(percresults[x], 2)}%)")
    else:
        print(f"{industry} (-{round(percresults[x], 2)}%)")
        
    industrystocks = stockdf2[stockdf2["Industry"] == industry].sort_values("Percent Change (Today)", ascending = False).reset_index().head(stockNumResults)

    print(format("TICKER", ">15"), end = "")
    print(format("PRICE", ">15"), end = "")
    print(format("PERCENT CHANGE", ">21"), end = "")
    print(format("P/E RATIO", ">15"))
    
    for y in range(stockNumResults):
        stockDisplay = industrystocks["Stock Tickers"][y]
        priceDisplay = industrystocks["Price"][y]
        percDisplay = industrystocks["Percent Change (Today)"][y]
        peDisplay = industrystocks["P/E Ratio"][y]
        
        print(format(f"{stockDisplay}", ">15"), end = "")
        print(format(f"${priceDisplay}", ">15"), end = "")
        print(format(f"{percDisplay}%", ">21"), end = "")
        print(format(f"{peDisplay}", ">15"))
        
        if y+2 > len(industrystocks["Stock Tickers"]):
            break

How many industries would you like me to display?10
How many stocks per industry would you like me to display?30
1: Oil & Gas E&P (+2.14%)
         TICKER          PRICE       PERCENT CHANGE      P/E RATIO
           PDCE         $46.99                7.71%         -34.81
            CEI          $2.04                7.37%          -1.36
            CRC         $41.29                6.28%           0.85
           NEXT           $2.8                5.66%          -8.28
            KRP         $13.94                5.61%          -6.58
            GTE          $0.67                5.44%          -1.17
            CPG          $4.23                5.29%           1.35
            BRY          $6.67                5.28%          -4.57
           TALO         $13.67                5.28%          -1.45
            CPE          $42.7                 5.2%          -1.41
            ERF          $7.18                4.82%          -5.98
            CNX         $11.64                4.66%      